## 有名モデル

- LeNet
- VGG

#### ライブラリ

In [1]:
import torch
from torch import nn
from torch.nn import functional as F
import torchvision
from torchvision import transforms
from torchvision import models
from torchvision.models.vgg import VGG11_Weights
from torch.utils.data import DataLoader, Subset

%load_ext autoreload
%autoreload 2
import utils

### LeNet

In [37]:
class LeNet(nn.Module):
    def __init__(self):
        super().__init__
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

### VGG11

In [3]:
# VGG11
model = models.vgg11(weights=VGG11_Weights.IMAGENET1K_V1)   # weightsで学習済みモデルを指定
# モデルの構造を確認
print("============feature============")
print(model.features)
# モデルの最終層を取得
print("============classifier============")
print(model.classifier)

============feature============
Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (4): ReLU(inplace=True)
  (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (7): ReLU(inplace=True)
  (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (9): ReLU(inplace=True)
  (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (12): ReLU(inplace=True)
  (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (14): ReLU(inplace=True)
  (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (16): Conv2d(512, 

In [4]:
# データ準備
trasform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

# データセットのダウンロード(cifar10)
class_names = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=trasform)
val_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=trasform) 

# データセットをサブセットにする
train_dataset_sub = Subset(train_dataset, range(1000))   # 1000枚のデータにする
val_dataset_sub = Subset(val_dataset, range(500))

# データローダーの作成
train_loader = DataLoader(train_dataset_sub, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset_sub, batch_size=32, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
# optimizer
opt = torch.optim.SGD(model.parameters(), lr=0.03)

In [6]:
# モデルの最終層を書き換え(10クラス分類に変更)
model.classifier[-1] = nn.Linear(4096, 10)

In [7]:
train_losses, val_losses, val_accuracies = utils.learn(model, train_loader, val_loader, opt, F.cross_entropy, epoch=3)

epoch: 0: train error: 2.0674091540277004, validation error: 2.1825543716549873, validation accuracy: 0.29453125037252903


epoch: 1: train error: 1.591926384717226, validation error: 2.228966251015663, validation accuracy: 0.2703125001862645


epoch: 2: train error: 1.3256622441112995, validation error: 4.351522535085678, validation accuracy: 0.25624999962747097
